In [1]:
if 'google.colab' in str(get_ipython()):
    %matplotlib inline
else:
    %matplotlib notebook

In [2]:
import time
from funtion_file import *

In [3]:
matrix_params = {
    'mat_size': (5, 5),
    'strength_kernel': (0.5, 3), 
    'min_max_line_size': [(1, 3), (1, 3)],
    'rotate': True,
    'fades_per_mat': 10,
    'new_background': True,
    'shape': 'line',  # 'line', 'triangle', 'face'
}

matrix_lister = MatrixLister(**matrix_params)

In [4]:
model, callbacks = matrix_lister.init_model(64, 128, 'cnn_gru')
matrix_lister.load_model(model, 'auto')

Loaded line weights


In [5]:
batch_size = 100
batch_num = 1
epochs = 5

generator = matrix_lister.init_generator(batch_size, batch_num)


start = time.time()
hist = model.fit(generator, epochs=epochs)
print(time.time() - start)

Epoch 1/5


1/1 [==============================] - 2s 2s/step - loss: 1.1444 - precision: 0.1080 - recall: 0.0400
Epoch 2/5
1/1 [==============================] - 0s 138ms/step - loss: 1.0753 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 3/5
1/1 [==============================] - 0s 133ms/step - loss: 1.0363 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 4/5
1/1 [==============================] - 0s 138ms/step - loss: 1.0030 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 5/5
1/1 [==============================] - 0s 140ms/step - loss: 0.9709 - precision: 0.0000e+00 - recall: 0.0000e+00
2.5860910415649414


In [6]:
matrix_lister.eval(model)

1/1 [==============================] - 0s 206ms/step
[<tf.Tensor: shape=(5,), dtype=float32, numpy=array([1., 1., 1., 1., 1.], dtype=float32)>, <tf.Tensor: shape=(5,), dtype=float32, numpy=array([1. , 1. , 0.8, 0.8, 0.8], dtype=float32)>, <tf.Tensor: shape=(5,), dtype=float32, numpy=array([1. , 1. , 0.8, 0.8, 0.8], dtype=float32)>, <tf.Tensor: shape=(5,), dtype=float32, numpy=array([1. , 1. , 0.8, 0.8, 0.8], dtype=float32)>, <tf.Tensor: shape=(5,), dtype=float32, numpy=array([1. , 1. , 0.8, 0.8, 0.8], dtype=float32)>, <tf.Tensor: shape=(5,), dtype=float32, numpy=array([1. , 1. , 0.8, 0.8, 0.8], dtype=float32)>, <tf.Tensor: shape=(5,), dtype=float32, numpy=array([1. , 1. , 0.8, 0.8, 0.8], dtype=float32)>, <tf.Tensor: shape=(5,), dtype=float32, numpy=array([1. , 1. , 0.8, 0.8, 0.8], dtype=float32)>, <tf.Tensor: shape=(5,), dtype=float32, numpy=array([1. , 1. , 0.8, 0.8, 0.8], dtype=float32)>, <tf.Tensor: shape=(5,), dtype=float32, numpy=array([1. , 1. , 0.8, 0.8, 0.8], dtype=float32)>]


In [ ]:
matrix_lister.save_model(model, 'auto')  # auto, line, triangle, none

In [ ]:
%matplotlib notebook
matrix_lister.display_frames(model, 16)

In [ ]:
ani = matrix_lister.plot_matrices(model, 50, interval=500)
plt.show()

In [ ]:
plot_training_history(hist, with_val=False)